# Deep Learning course 2022-2023
## Project: Video Prediction on Moving MNIST
### Project Contributors
* Mattia Castelmare, 1815675
* Andrea Giuseppe Di Francesco, 1836928
* Enrico Fazzi, 2003876

#### Installation cells

In [ ]:
#!git clone https://github.com/tychovdo/MovingMNIST.git
#!pip3 install pytorch-lightning==1.5.10
# !pip3 install torchvision
# !pip3 install matplotlib
# !pip install wandb



### Importing Libraries

In [ ]:
### wandb codes ###
import wandb
#####################

import torch
import os
import os
import torch
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import Callback
from pytorch_lightning import Trainer
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')


from MovingMNIST.MovingMNIST import *

## Useful Functions

In [ ]:
def show_video(tensor):
    ''' This function display a video, given a torch tensor (source: https://stackoverflow.com/questions/67261108/how-to-display-a-video-in-colab-using-a-pytorch-tensor-of-rgb-image-arrays)
        INPUT: tensor (Frames x Channels x Height x Width) 
        OUTPUT: Display an animation '''

    fig, ax = plt.subplots()

    imgs = tensor
    imgs = imgs.permute(0, 2, 3, 1)  # Permuting to (Bx)HxWxC format
    frames = [[ax.imshow(imgs[i], cmap='gray')] for i in range(len(imgs))]

    ani = animation.ArtistAnimation(fig, frames)
    return ani


def collate(batch):
    ''' This function is used within the Dataloader pytorch utility. 
        INPUT: batch: List of tuples of tensor, each tuple contains the input video and the target video, 
        OUTPUT: A tuple (input, gt), s.t. both of them has dimension (B x T x C x H x W)'''

    list_tuples = list(map(lambda x: (torch.reshape(x[0], (1, x[0].shape[0], 1, x[0].shape[1], x[0].shape[2])), torch.reshape(
        x[1], (1, x[1].shape[0], 1, x[1].shape[1], x[1].shape[2]))), batch))
    
    input = list_tuples[0][0]
    gt = list_tuples[0][1]
    
    for i in range(1, len(list_tuples)):
        input = torch.cat((input, list_tuples[i][0]), dim=0)
        gt = torch.cat((gt, list_tuples[i][1]), dim = 0)
    
    return (input.type(torch.FloatTensor), gt.type(torch.FloatTensor))    



### Useful variables 

In [ ]:
shuffle_train = True
shuffle_test = False
download = False

### wandb codes ###
wb = True
project_name = "VP_on_MMNIST"
if wb:
    wandb.login()
###################

SEED = 2812023
pl.seed_everything(SEED)

root = './data'
if not os.path.exists(root):
    os.mkdir(root)


### Pytorch lightning DataModule:
* Set the MMNIST Dataset object and Dataloader 

In [ ]:
class pl_Dataset(pl.LightningDataModule):

    def __init__(self, batch_size):
         
         self.bs = batch_size


    def setup(self, stage = None):
        if stage == 'fit':
            self.train_set = MovingMNIST(root=root,
                        train=True,
                        download = download)
        elif stage == 'test':
            self.test_set = MovingMNIST(root=root,
                        train=False,
                        download = download)
            

    def train_dataloader(self, *args, **kwargs):

        train_loader = torch.utils.data.DataLoader(
                 dataset = self.train_set,
                 batch_size=self.bs,
                 shuffle=shuffle_train,
                 collate_fn = collate)

        return train_loader

    def val_dataloader(self, *args, **kwargs):

        test_loader = torch.utils.data.DataLoader(
                 dataset = self.test_set,
                 batch_size=self.bs,
                 shuffle=shuffle_test,
                 collate_fn = collate)

        return test_loader
    




### Hyperparameters

In [ ]:
batch_size = 128
lr = 1e-2
wd = 5e-2

tau = 1e-1 # KL Divergency parameter


hyperparameters = {'lr': lr,
                'wd': wd,
                'epochs': 100,
                'batch_size': batch_size
                }

data = pl_Dataset(batch_size)

data.setup(stage = 'fit')
data.setup(stage = 'test')



### Models

In [ ]:
class Encoder(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(Encoder, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * 2, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
    
    def forward(self, x):
        return self.conv(x)

class Decoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Decoder, self).__init__()
        self.conv = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(out_channels, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU()
        )
    
    def forward(self, x):
        return self.conv(x)

class RandomBaseline(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(RandomBaseline, self).__init__()
        self.encoder = Encoder(in_channels, out_channels)
        self.decoder = Decoder(out_channels * 2, out_channels)
    
    def forward(self, x):
  
        original_shape = x.shape
     
        x = x.reshape((original_shape[0]*original_shape[1], original_shape[2], original_shape[3], original_shape[4]))


        x = self.encoder(x)
   
        
        x = self.decoder(x)
     
        x = x.reshape((original_shape[0], original_shape[1], original_shape[2], original_shape[3], original_shape[4]))
        

        return x
        
    

In [ ]:
class plTrainingModule(pl.LightningModule):
    
    def __init__(self, model):
        super(plTrainingModule, self).__init__()
        self.model = model
        self.MSE = nn.MSELoss()
        self.KL = nn.KLDivLoss()
        self.tot_loss_tr = []
        self.tot_loss = []

        self.KL_list = []
        self.mse_list = []

    def training_step(self, batch, batch_idx):
        
        ground_truth = batch[1]
       
        ### OUTPUT COMPUTATION ###
       
        prediction = self.model(batch[0])
        KL_Loss = self.compute_KLloss(prediction, ground_truth)
        mse = self.MSE(prediction, ground_truth)

        loss = mse + KL_Loss
 
        self.tot_loss_tr.append(loss.item())
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        
        ground_truth = batch[1]
        
        ### OUTPUT COMPUTATION ###

        prediction = self.model(batch[0])

        KL_Loss = self.compute_KLloss(prediction, ground_truth)

        mse = self.MSE(prediction, ground_truth)
        
        loss = mse + KL_Loss
        
        self.tot_loss.append(loss.item())
        self.KL_list.append(KL_Loss.item())
        self.mse_list.append(mse.item())



        return loss
    

    def compute_KLloss(self, predicted, ground_truth):

        delta_pred = (predicted[:, 1:] - predicted[:, :-1])/tau
        delta_gt = (ground_truth[:, 1:] - ground_truth[:, :-1])/tau

        soft_pred = F.softmax(delta_pred, dim = 2) # 2 is the channel-related dimension
        soft_gt = F.softmax(delta_gt, dim = 2) # 2 is the channel-related dimension



        KL_Loss = self.KL(soft_pred, soft_gt)

        return KL_Loss

    def on_epoch_end(self):
        if len(self.tot_loss_tr)!= 0:
            loss_train_mean = sum(self.tot_loss_tr)/len(self.tot_loss_tr) 
            loss_mean = sum(self.tot_loss)/len(self.tot_loss)
            KL_mean = sum(self.KL_list)/len(self.KL_list)
            mse_mean = sum(self.mse_list)/len(self.mse_list)

            self.log(name = 'TOT. Loss on train', value = loss_train_mean)
            self.log(name = 'TOT. Loss on test', value = loss_mean)
            self.log(name = 'KL loss on test', value = KL_mean)
            self.log(name = 'MSE loss on test', value = mse_mean)


    def configure_optimizers(self):
        return torch.optim.AdamW(self.model.parameters(), lr, weight_decay = wd)

In [ ]:
num_gpu = 1 if torch.cuda.is_available() else 0

model_conf = 'random'
exp_name = model_conf + ' ' + str(hyperparameters['epochs']) + ' ' + str(batch_size)

In [ ]:
model = RandomBaseline(1, 64)


pl_training_MDL = plTrainingModule(model)


### WANDB CODE ###
if wb:

    wandb_logger = WandbLogger(project=project_name, name = exp_name, config = hyperparameters, entity = 'deepl_wizards')
    trainer = pl.Trainer(
            max_epochs = hyperparameters['epochs'],  # maximum number of epochs.
            gpus=num_gpu,  # the number of gpus we have at our disposal.
            default_root_dir="", logger = wandb_logger        #, overfit_batches = 1
        )

else:
    trainer = pl.Trainer(
        max_epochs = hyperparameters['epochs'],  # maximum number of epochs.
        gpus=num_gpu,  # the number of gpus we have at our disposal.
        default_root_dir=""
    )



In [ ]:
trainer.fit(model = pl_training_MDL, datamodule = data)

### WANDB code
wandb.finish()

In [ ]:
#SOFTMAX ISSUES

delta_pred = (input[:, 1:, :, :, :] - input[:, :-1, : , :, :])/tau 
# delta_pred += 1200
delta_gt = (gt[:, 1:, :, :, :] - gt[:, :-1, :, :, :])/tau
# delta_gt += 1200
soft_pred = F.softmax(delta_pred, dim = 2) # 2 is the channel-related dimension
soft_gt = F.softmax(delta_gt, dim = 2) 

In [ ]:
torch.equal(delta_pred.to('cuda'), delta_gt.to('cuda'))
torch.equal(soft_pred.to('cuda'), soft_gt.to('cuda'))


In [ ]:
for batch in data.train_dataloader():
    gt = batch[1]
    input = model(batch[0])
    break

In [ ]:
show_video(gt[0])

In [ ]:
show_video(input[0].detach())
